In [98]:
import random
(q0, q1, q2) = (7, 11, 13) # RNS basis of size 3
l = [q0, q1, q2]
q=1
p1 = 17 # p1 is the message modulus
p2 = 3
#print(p1*p2)
for x in l:
    q = q*x

# q is ciphertext modulus. 391 = p1.p2 < q = 1001
#print(q)
Z = Integers()
Zq = Integers(q)
Zq0 = Integers(q0)
Zq1 = Integers(q1)
Zq2 = Integers(q2)
Zp1 = Integers(p1)
# Define the polynomial rings
Rm.<x> = PolynomialRing(Zp1, 'x')
R.<x> = PolynomialRing(Zq, 'x')
R0.<x> = PolynomialRing(Zq0, 'x')
R1.<x> = PolynomialRing(Zq1, 'x')
R2.<x> = PolynomialRing(Zq2, 'x')

# Define the quotient ring Z_q[x]/(x^n + 1)
n = 4
Qmod = R.quotient(x^n + 1, 'x')
Q0 = R0.quotient(x^n + 1, 'x')
Q1 = R1.quotient(x^n + 1, 'x')
Q2 = R2.quotient(x^n + 1, 'x')
Rmod = Rm.quotient(x^n + 1, 'x')

q0_t = q//q0
q1_t = q//q1
q2_t = q//q2

q0_t_inverse = Zq0(1)/Zq0(q0_t)
q1_t_inverse = Zq1(1)/Zq1(q1_t)
q2_t_inverse = Zq2(1)/Zq2(q2_t)


crt_moduli = [p1, p2] # CRT modulus 

e_0 = Qmod(crt([0,1],crt_moduli) + crt([0,1],crt_moduli)*Qmod(x^2) + crt([0,1],crt_moduli)*Qmod(x^5) + crt([0,1],crt_moduli)*Qmod(x^7) ) # This is encode(0, 1 + x^2 + x^5 + x^7)
e_1 = Qmod(crt([0,1],crt_moduli) + crt([0,1],crt_moduli)*Qmod(x) + crt([0,1],crt_moduli)*Qmod(x^4) + crt([0,1],crt_moduli)*Qmod(x^6) )
e_2 = Qmod(crt([0,1],crt_moduli) + crt([0,1],crt_moduli)*Qmod(x^3) + crt([0,1],crt_moduli)*Qmod(x^6) + crt([0,1],crt_moduli)*Qmod(x^7) )

random_polynomial = sum([Zq(random.randint(0, q-1))*Qmod(x^i) for i in range(n)])
alpha_0 = Qmod(random_polynomial)
random_polynomial = sum([Zq(random.randint(0, q-1))*Qmod(x^i) for i in range(n)])
alpha_1 = Qmod(random_polynomial)
random_polynomial = sum([Zq(random.randint(0, q-1))*Qmod(x^i) for i in range(n)])
alpha_2 = Qmod(random_polynomial)

s = Qmod(1 + x + x^3 + x^5 + x^6 + x^7 ) # s is the secret key

#print(s)
# Generation of public key:

random_polynomial = sum([Zq(random.randint(0, 2))*Qmod(x^i) for i in range(n)])
mask = Qmod(random_polynomial)
e = Qmod(crt([0,1],crt_moduli) + crt([0,1],crt_moduli)*Qmod(x) + crt([0,1],crt_moduli)*Qmod(x^2) + crt([0,1],crt_moduli)*Qmod(x^3) + crt([0,1],crt_moduli)*Qmod(x^5) + crt([0,1],crt_moduli)*Qmod(x^7) )

public_0 = Qmod(mask*s + e)
public_1 = Qmod(-(mask))

beta_0 = Qmod(Zq(q0_t)*Zq(q0_t_inverse)*s*s - alpha_0*s + e_0 )
beta_1 = Qmod(Zq(q1_t)*Zq(q1_t_inverse)*s*s - alpha_1*s + e_1)
beta_2 = Qmod(Zq(q2_t)*Zq(q2_t_inverse)*s*s - alpha_2*s + e_2)

#Reduction of beta_0, beta_1, beta_2, alpha_0, alpha_1, alpha_2 in RNS basis [q0, q1, q2]:

def reduce_coefficients(f,p):
    f_parent = f.lift()
    coeffs = f_parent.coefficients(sparse=False)
    reduced_coeffs = [Z(c) % p for c in coeffs]
    f_reduced = sum(reduced_coeffs[i] * x^i for i in range(len(reduced_coeffs)))
    return f_reduced

beta_00 = Q0(reduce_coefficients(beta_0, q0))
beta_01 = Q1(reduce_coefficients(beta_0, q1))
beta_02 = Q2(reduce_coefficients(beta_0, q2))

beta_10 = Q0(reduce_coefficients(beta_1, q0))
beta_11 = Q1(reduce_coefficients(beta_1, q1))
beta_12 = Q2(reduce_coefficients(beta_1, q2))

beta_20 = Q0(reduce_coefficients(beta_2, q0))
beta_21 = Q1(reduce_coefficients(beta_2, q1))
beta_22 = Q2(reduce_coefficients(beta_2, q2))

alpha_00 = Q0(reduce_coefficients(alpha_0, q0))
alpha_01 = Q1(reduce_coefficients(alpha_0, q1))
alpha_02 = Q2(reduce_coefficients(alpha_0, q2))

alpha_10 = Q0(reduce_coefficients(alpha_1, q0))
alpha_11 = Q1(reduce_coefficients(alpha_1, q1))
alpha_12 = Q2(reduce_coefficients(alpha_1, q2))

alpha_20 = Q0(reduce_coefficients(alpha_2, q0))
alpha_21 = Q1(reduce_coefficients(alpha_2, q1))
alpha_22 = Q2(reduce_coefficients(alpha_2, q2))


In [99]:
def generate_encryption(m):
    l = [0,1]
    random_polynomial = sum([Zq(random.randint(0, 2))*Qmod(x^i) for i in range(n)])
    u = Qmod(random_polynomial)
    random_crt_polynomial = sum([Zq(crt([m[i], random.choice(l)],crt_moduli))*Qmod(x^i) for i in range(n)])
    encode1 = Qmod(random_crt_polynomial)
    random_crt_polynomial = sum([Zq(crt([0,random.choice(l)],crt_moduli))*Qmod(x^i) for i in range(n)])
    encode2 = Qmod(random_crt_polynomial)
    
    c_0 = Qmod(public_0*u + encode1)
    c_1 = Qmod(public_1*u + encode2)
    return (c_0, c_1)

def decryption(c_0, c_1):
    reduced_coeffs = []
    T = Qmod(c_0 + c_1*s)
    T_parent = T.lift()
    coeffs = T_parent.coefficients(sparse=False)
    for c in coeffs:
        if Z(c)<q//2:
            reduced_coeffs.append(Z(c) % p1)
        else:
            reduced_coeffs.append(Z(q - c) % p1)
    #reduced_coeffs = [Z(c) % p1 for c in coeffs]
    T_reduced = sum(reduced_coeffs[i] * Rmod(x^i) for i in range(len(reduced_coeffs)))
    return T_reduced



In [101]:
def multiplication(a, b, c, d): # (a, b) and (c, d) are 1st and 2nd ciphertexts respectively to multiply by relinearization
    c_0 = a*b
    c_1 = a*d + b*c
    c_2 = b*d
    
    c_20 = Qmod(reduce_coefficients(h_2, q0))
    c_21 = Qmod(reduce_coefficients(h_2, q1))
    c_22 = Qmod(reduce_coefficients(h_2, q2))
    
    c_0_tilda = beta_0*h_20 + beta_1*h_21 + beta_2*h_22
    c_1_tilda = alpha_0*h_20 + alpha_1*h_21 + alpha_2*h_22
    
    return (c_0 + c_0_tilda, c_1 + c_1_tilda)

#Relinearization in RNS basis [q0, q1, q2]:

def multiplication_in_RNSbasis(a, b, c, d):
    c_0 = a*b
    c_1 = a*d + b*c
    c_2 = b*d
    
    c_20 = Qmod(reduce_coefficients(c_2, q0))
    c_21 = Qmod(reduce_coefficients(c_2, q1))
    c_22 = Qmod(reduce_coefficients(c_2, q2))
    
    c_200 = Q0(reduce_coefficients(c_20, q0))
    c_201 = Q1(reduce_coefficients(c_20, q1))
    c_202 = Q2(reduce_coefficients(c_20, q2))
    
    c_210 = Q0(reduce_coefficients(c_21, q0))
    c_211 = Q1(reduce_coefficients(c_21, q1))
    c_212 = Q2(reduce_coefficients(c_21, q2))
    
    c_220 = Q0(reduce_coefficients(c_22, q0))
    c_221 = Q1(reduce_coefficients(c_22, q1))
    c_222 = Q2(reduce_coefficients(c_22, q2))
    
    # L0, L1, L2 are components of final RNS tuple which is RNS representation of c_0_tilda
    L0 = Qmod(reduce_coefficients(beta_00*c_200 + beta_10*c_210 + beta_20*c_220, q)) 
    L1 = Qmod(reduce_coefficients(beta_01*c_201 + beta_11*c_211 + beta_21*c_221, q))
    L2 = Qmod(reduce_coefficients(beta_02*c_202 + beta_12*c_212 + beta_22*c_222, q))
    # K0, K1, K2 are components of final RNS tuple which is RNS representation of c_1_tilda
    K0 = Qmod(reduce_coefficients(alpha_00*c_200 + alpha_10*c_210 + alpha_20*c_220, q))
    K1 = Qmod(reduce_coefficients(alpha_01*c_201 + alpha_11*c_211 + alpha_21*c_221, q))
    K2 = Qmod(reduce_coefficients(alpha_02*c_202 + alpha_12*c_212 + alpha_22*c_222, q))
    
    c_0_tilda = Qmod(Zq(q0_t)*Zq(q0_t_inverse)*L0 + Zq(q1_t)*Zq(q1_t_inverse)*L1 + Zq(q2_t)*Zq(q2_t_inverse)*L2)
    c_1_tilda = Qmod(Zq(q0_t)*Zq(q0_t_inverse)*K0 + Zq(q1_t)*Zq(q1_t_inverse)*K1 + Zq(q2_t)*Zq(q2_t_inverse)*K2)
    
    return (c_0 + c_0_tilda, c_1 + c_1_tilda)



In [105]:

m_1 = [7, 12, 3, 14]
m_2 = [0, 12, 4, 2]
m_1poly = Rmod(sum(m_1[i]*Rmod(x^i) for i in range(n)))
m_2poly = Rmod(sum(m_2[i]*Rmod(x^i) for i in range(n)))
print(m_1poly,'\n', m_2poly, '\n m_1*m_2=',m_1poly*m_2poly)

(a_0, b_0) = generate_encryption(m_1)
(c_0, d_0) = generate_encryption(m_2)
print("Encryption of m_1:", a_0, "and", b_0)
print("Encryption of m_2:", c_0, "and", d_0)
print("Decryption for m_1:",decryption(a_0, b_0))
print("Decryption for m_2:",decryption(c_0, d_0))
(v_0, v_1) = multiplication_in_RNSbasis(a_0, b_0, c_0, d_0)
print("Multiplication of encrypted m_1 and encrypted m_2:",multiplication_in_RNSbasis(a_0, b_0, c_0, d_0))
print("Decryption of multiplication:",decryption(v_0, v_1))

14*x^3 + 3*x^2 + 12*x + 7 
 2*x^3 + 4*x^2 + 12*x 
 m_1*m_2= 13*x^3 + 8*x^2 + 5*x
Encryption of m_1: 122*x^3 + 179*x^2 + 50*x + 22 and 996*x^3 + 32*x^2 + 35*x + 4
Encryption of m_2: 180*x^3 + 113*x^2 + 48*x + 962 and 29*x^3 + 33*x^2 + 3*x + 6
Decryption for m_1: 14*x^3 + 3*x^2 + 12*x + 7
Decryption for m_2: 2*x^3 + 4*x^2 + 12*x
Multiplication of encrypted m_1 and encrypted m_2: (248*x^3 + 219*x^2 + 188*x + 823, 118*x^3 + 614*x^2 + 928*x + 429)
Decryption of multiplication: 14*x^3 + 13*x^2 + 12*x


Results (for dimention 2^2 = 4):

Output 1:
 m_1 =   11*x^3 + 4*x^2 + 11 
 m_2 = 7*x + 1 
 m_1*m_2= 5*x^3 + 4*x^2 + 9*x + 2
Encryption of m_1: 185*x^3 + 66*x^2 + 28*x + 94 and 18*x^3 + 38*x^2 + 989*x + 999
Encryption of m_2: 70*x^3 + 144*x^2 + 106*x + 34 and 993*x^3 + 22*x^2 + 28*x + 30
Decryption for m_1: 11*x^3 + 4*x^2 + 11
Decryption for m_2: 7*x + 1
Multiplication of encrypted m_1 and encrypted m_2: (233*x^3 + 817*x^2 + 850*x + 33, 47*x^3 + 762*x^2 + 52*x + 544)
Decryption of multiplication: 7*x^3 + 14*x + 15

    
Output 2:

m_1 = 2*x^3 + 9*x^2 + 4*x + 5 
 m_2 = 16*x^3 + 11*x^2 + 13*x + 9 
 m_1*m_2= 4*x^3 + 3*x^2 + 3*x + 9
Encryption of m_1: 116*x^3 + 15*x^2 + 53*x + 31 and 29*x^3 + 35*x^2 + 24*x + 7
Encryption of m_2: 156*x^3 + 107*x^2 + 58*x + 75 and 18*x^3 + 4*x^2 + 22*x + 32
Decryption for m_1: 2*x^3 + 9*x^2 + 4*x + 5
Decryption for m_2: 16*x^3 + 11*x^2 + 13*x + 9
Multiplication of encrypted m_1 and encrypted m_2: (689*x^3 + 401*x^2 + 107*x + 541, 928*x^3 + 771*x^2 + 401*x + 562)
Decryption of multiplication: 11*x^3 + 15*x^2 + 10*x + 6

Output 3:

x^3 + x^2 + x + 1 
 16*x^3 + 11*x^2 + 13*x + 12 
 m_1*m_2= x^3 + 3*x^2 + 15*x + 6
Encryption of m_1: 158*x^3 + 63*x^2 + 29*x + 84 and 985*x^3 + 38*x^2 + 22*x + 999
Encryption of m_2: 173*x^3 + 107*x^2 + 58*x + 78 and 985*x^3 + 38*x^2 + 989*x + 32
Decryption for m_1: x^3 + x^2 + x + 1
Decryption for m_2: 16*x^3 + 11*x^2 + 13*x + 12
Multiplication of encrypted m_1 and encrypted m_2: (563*x^3 + 724*x^2 + 733*x + 675, 529*x^3 + 791*x^2 + 940*x + 476)
Decryption of multiplication: 16*x^3 + 4*x^2 + 13*x + 6

Output 4:

5*x^3 + 4*x^2 + 9*x + 2 
 x^3 + 3*x^2 + 15*x + 6 
 m_1*m_2= 7*x^2 + 14*x + 1
Encryption of m_1: 183*x^3 + 86*x^2 + 59*x + 967 and 22*x^3 + 995*x^2 + 997*x + 42
Encryption of m_2: 100*x^3 + 38*x^2 + 990*x + 970 and 990*x^3 + 35*x^2 + 37*x + 2
Decryption for m_1: 5*x^3 + 4*x^2 + 9*x + 2
Decryption for m_2: x^3 + 3*x^2 + 15*x + 6
Multiplication of encrypted m_1 and encrypted m_2: (976*x^3 + 352*x^2 + 201*x + 425, 928*x^3 + 440*x^2 + 988*x + 551)
Decryption of multiplication: 15*x^3 + 3*x^2 + 13*x + 7

Output 5:

0 
 x^3 + 14*x^2 + 7*x 
 m_1*m_2= 0
Encryption of m_1: 112*x^3 + 997*x^2 + 977*x + 993 and 24*x^3 + 40*x^2 + 36
Encryption of m_2: 96*x^3 + 27*x^2 + 967*x + 993 and 991*x^3 + 40*x^2 + 2
Decryption for m_1: 0
Decryption for m_2: x^3 + 14*x^2 + 5*x
Multiplication of encrypted m_1 and encrypted m_2: (160*x^3 + 980*x^2 + 903*x + 342, 230*x^3 + 738*x^2 + 489*x + 685)
Decryption of multiplication: x^3 + 15*x^2 + 9*x + 16

Output 6:

0 
 8*x^3 + 14*x^2 + 7*x + 1 
 m_1*m_2= 0
Encryption of m_1: 86*x^3 + 74*x^2 + 973*x + 935 and 989*x^3 + 997*x^2 + 40*x + 36
Encryption of m_2: 142*x^3 + 138*x^2 + 5*x + 990 and 986*x^3 + 29*x^2
Decryption for m_1: 15
Decryption for m_2: 8*x^3 + 14*x^2 + 5*x + 1
Multiplication of encrypted m_1 and encrypted m_2: (468*x^3 + 809*x^2 + 649*x + 812, 880*x^3 + 76*x^2 + 264*x + 695)
Decryption of multiplication: 15*x^3 + 3*x^2 + 10*x + 4

Output 7:

10*x^3 + 10*x^2 + 10*x + 10 
 7*x^3 + 7*x^2 + 7*x + 7 
 m_1*m_2= 8*x^3 + 4*x^2 + 13
Encryption of m_1: 53*x^3 + 66*x^2 + 6*x + 84 and 23*x^3 + 3*x^2 + 32*x + 25
Encryption of m_2: 7*x^3 + 7*x^2 + 7*x + 7 and 34*x^3 + 34*x
Decryption for m_1: 10*x^3 + 10*x^2 + 10*x + 10
Decryption for m_2: 7*x^3 + 7*x^2 + 7*x + 7
Multiplication of encrypted m_1 and encrypted m_2: (67*x^3 + 537*x^2 + 876*x + 958, 536*x^3 + 53*x^2 + 869*x + 743)
Decryption of multiplication: 4*x^3 + 15*x^2 + 7*x + 5

Output 8:

11*x^3 + 8*x^2 + 15*x + 6 
 2*x^3 + 13*x^2 + 11*x + 1 
 m_1*m_2= 8*x^2 + 7*x + 6
Encryption of m_1: 119*x^3 + 122*x^2 + 57*x + 979 and 994*x^3 + 992*x^2 + 1000*x + 37
Encryption of m_2: 61*x^3 + 68*x^2 + 15*x + 970 and 28*x^3 + 31*x^2 + 2*x + 35
Decryption for m_1: 11*x^3 + 8*x^2 + 15*x + 6
Decryption for m_2: 2*x^3 + 13*x^2 + 11*x + 1
Multiplication of encrypted m_1 and encrypted m_2: (845*x^3 + 430*x^2 + 513*x + 56, 602*x^3 + 325*x^2 + 350*x + 976)
Decryption of multiplication: 11*x^3 + 15*x^2 + 5*x + 16


Results (for dimention n = 2^1 = 2):

Output 1:

12*x + 10 
 11*x + 7 
 m_1*m_2= 7*x + 6
Encryption of m_1: 12*x + 27 and 25*x + 2
Encryption of m_2: 28*x + 7 and 34*x
Decryption for m_1: 12*x + 10
Decryption for m_2: 11*x + 7
Multiplication of encrypted m_1 and encrypted m_2: (495*x + 162, 821*x + 80)
Decryption of multiplication: 2*x + 9

Output 2:

13*x + 11 
 7*x + 6 
 m_1*m_2= 2*x + 9
Encryption of m_1: 30*x + 28 and 999*x + 8
Encryption of m_2: 24*x + 6 and 25*x + 2
Decryption for m_1: 13*x + 11
Decryption for m_2: 7*x + 6
Multiplication of encrypted m_1 and encrypted m_2: (422*x + 46, 635*x + 578)
Decryption of multiplication: 14*x + 12

Output 3:

x 
 0 
 m_1*m_2= 0
Encryption of m_1: x and 991*x + 6
Encryption of m_2: 34*x and 996*x + 3
Decryption for m_1: x
Decryption for m_2: 0
Multiplication of encrypted m_1 and encrypted m_2: (769*x + 180, 840*x + 358)
Decryption of multiplication: 4*x + 10

Output 4:

2*x + 9 
 16*x + 11 
 m_1*m_2= 13*x + 16
Encryption of m_1: 36*x + 43 and 991*x + 40
Encryption of m_2: 33*x + 28 and 997*x + 33
Decryption for m_1: 2*x + 9
Decryption for m_2: 16*x + 11
Multiplication of encrypted m_1 and encrypted m_2: (908*x + 316, 179*x + 924)
Decryption of multiplication: 7*x + 10

Results (for dimention 2^3 = 8):

Output 1:

15*x^7 + 13*x^5 + x^4 + 11*x^3 + 13*x^2 + 2*x + 9 
 13*x^7 + 11*x^6 + 9*x^5 + 7*x^4 + x^3 + x^2 + 16*x + 11 
 m_1*m_2= 2*x^7 + 12*x^6 + 11*x^5 + 12*x^4 + 11*x^3 + 16*x + 13
Encryption of m_1: 327*x^7 + 73*x^6 + 71*x^5 + 40*x^4 + 983*x^3 + 967*x^2 + 762*x + 794 and 991*x^7 + 977*x^6 + 24*x^5 + 29*x^4 + 33*x^3 + 47*x^2 + 59*x + 29
Encryption of m_2: 196*x^7 + 144*x^6 + 107*x^5 + 13*x^4 + 54*x^3 + 977*x^2 + 885*x + 913 and 982*x^7 + 979*x^6 + 984*x^5 + 999*x^4 + 993*x^3 + 33*x^2 + 48*x + 51
Decryption for m_1: 15*x^7 + 13*x^5 + x^4 + 11*x^3 + 11*x^2 + 7
Decryption for m_2: 13*x^7 + 11*x^6 + 9*x^5 + 7*x^4 + x^3 + x^2 + 14*x + 9
Multiplication of encrypted m_1 and encrypted m_2: (101*x^7 + 182*x^6 + 894*x^5 + 303*x^4 + 326*x^3 + 972*x^2 + 552*x + 760, 486*x^7 + 275*x^6 + 322*x^5 + 403*x^4 + 952*x^3 + 943*x^2 + 253*x + 606)
Decryption of multiplication: 13*x^7 + 14*x^5 + 12*x^4 + 10*x^3 + 5*x^2 + 7*x + 4

Output 2:

2*x^7 + 12*x^6 + 11*x^5 + 12*x^4 + 11*x^3 + 16*x + 13 
 0 
 m_1*m_2= 0
Encryption of m_1: 197*x^7 + 90*x^6 + 139*x^5 + 54*x^4 + 63*x^3 + 980*x^2 + 990*x + 989 and 984*x^7 + 987*x^6 + 992*x^5 + 993*x^4 + 992*x^3 + 43*x^2 + 37*x + 36
Encryption of m_2: 198*x^7 + 209*x^6 + 4*x^5 + 132*x^4 + 991*x^3 + 986*x^2 + 843*x + 747 and 970*x^7 + 971*x^6 + 990*x^5 + 20*x^4 + 29*x^3 + 3*x^2 + 19*x + 25
Decryption for m_1: 2*x^7 + 12*x^6 + 11*x^5 + 12*x^4 + 11*x^3 + 16*x + 11
Decryption for m_2: 15*x + 15
Multiplication of encrypted m_1 and encrypted m_2: (59*x^7 + 645*x^6 + 490*x^5 + 764*x^4 + 592*x^3 + 949*x^2 + 855*x + 595, 678*x^7 + 486*x^6 + 612*x^5 + 131*x^4 + 772*x^3 + 488*x^2 + 201*x + 463)
Decryption of multiplication: 11*x^7 + 6*x^6 + 2*x^5 + 7*x^4 + 4*x^2 + 14*x + 3

Output 3:


